<a href="https://colab.research.google.com/github/KuNNaL2412/Text_Summarization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing required Dependencies
!pip install datasets
!pip install transformers
!pip install rouge
!pip install rouge_score
!pip install tokenizers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=c69d8f1a7c0ad799701f50

In [2]:
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from rouge_score import rouge_scorer

# Loading CNN Dailymail Dataset and using only first 10 entries
dataset = datasets.load_dataset("cnn_dailymail", "3.0.0")
dataset = dataset["test"].select(range(10))

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
# MODEL - 1 facebook/bart-large-cnn-------------------------------------------------------------------------------------------

# Using Huggingface Pipeline for summarization models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print("Model-1 : facebook/bart-large-cnn")
for example in dataset:
  # Accessing every Article and Summary from the dataset
  article = example["article"]
  summary = example["highlights"]

  # Generating Summary using the pipeline with max & min length of 150 & 30 respectively
  generated_summary = summarizer(article, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]
  print("Dataset Summary: ", summary)
  print("Generated Summary: ", generated_summary)

  # Initializing ROUGE Scorer for evaluation
  rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  # Calculating ROUGE scores of Dataset Summary and Generated Summary
  scores = rouge.score(summary, generated_summary)

  # Printing Precision, Recall & F1 scores of ROUGE-1, ROUGE-2 & ROUGE-L
  print("--------------------------------------------------------------------------------------------------------------------------")
  print("ROUGE-1:", scores['rouge1'])
  print("ROUGE-2:", scores['rouge2'])
  print("ROUGE-L:", scores['rougeL'])
  print("--------------------------------------------------------------------------------------------------------------------------")


Model-1 : facebook/bart-large-cnn
Dataset Summary:  Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated Summary:  The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
--------------------------------------------------------------------------------------------------------------------------
ROUGE-1: Score(precision=0.5135135135135135, recall=0.5588235294117647, fmeasure=0.5352112676056339)
ROUGE-2: Score(precision=0.3611111111111111, recall=0.3939393939393939, fmeasure=0.37681159420289856)
ROUGE-L: Score(precision=0.4594594594594595, recall=0.5, fmeasure=0.47887323943661975)
--------------------------

Your max_length is set to 150, but your input_length is only 143. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=71)


Dataset Summary:  Once a super typhoon, Maysak is now a tropical storm with 70 mph winds .
It could still cause flooding, landslides and other problems in the Philippines .
Generated Summary:  Maysak gained super typhoon status just a few days ago. It has since lost steam as it has spun west in the Pacific Ocean. It boasts steady winds of more than 70 mph.
--------------------------------------------------------------------------------------------------------------------------
ROUGE-1: Score(precision=0.30303030303030304, recall=0.38461538461538464, fmeasure=0.3389830508474576)
ROUGE-2: Score(precision=0.09375, recall=0.12, fmeasure=0.10526315789473684)
ROUGE-L: Score(precision=0.18181818181818182, recall=0.23076923076923078, fmeasure=0.2033898305084746)
--------------------------------------------------------------------------------------------------------------------------
Dataset Summary:  Bob Barker returned to host "The Price Is Right" on Wednesday .
Barker, 91, had retired as hos

In [4]:
# MODEL - 2 Einmalumdiewelt/T5-Base_GNAD-------------------------------------------------------------------------------------------

# Using Huggingface Pipeline for summarization models
summarizer = pipeline("summarization", model="Einmalumdiewelt/T5-Base_GNAD")

print("Model-2 : Einmalumdiewelt/T5-Base_GNAD")
for example in dataset:
  # Accessing every Article and Summary from the dataset
  article = example["article"]
  summary = example["highlights"]

  # Generating Summary using the pipeline with max & min length of 150 & 30 respectively
  generated_summary = summarizer(article, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]
  print("Dataset Summary: ", summary)
  print("Generated Summary: ", generated_summary)

  # Initializing ROUGE Scorer for evaluation
  rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  # Calculating ROUGE scores of Dataset Summary and Generated Summary
  scores = rouge.score(summary, generated_summary)

  # Printing Precision, Recall & F1 scores of ROUGE-1, ROUGE-2 & ROUGE-L
  print("--------------------------------------------------------------------------------------------------------------------------")
  print("ROUGE-1:", scores['rouge1'])
  print("ROUGE-2:", scores['rouge2'])
  print("ROUGE-L:", scores['rougeL'])
  print("--------------------------------------------------------------------------------------------------------------------------")

Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors


Model-2 : Einmalumdiewelt/T5-Base_GNAD
Dataset Summary:  Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated Summary:  The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, giving the court jurisdiction over alleged crimes committed in Palestinian territories. Israel and the United States, neither of which is an ICC member, oppose the move.
--------------------------------------------------------------------------------------------------------------------------
ROUGE-1: Score(precision=0.5121951219512195, recall=0.6176470588235294, fmeasure=0.56)
ROUGE-2: Score(precision=0.35, recall=0.42424242424242425, fmeasure=0.3835616438356164)
ROUGE-L: Score(precision=0.43902439024390244, recall=0.5294117647058824, fmeasure=0.48)
----------------------

In [5]:
# MODEL - 3 google/pegasus-cnn_dailymail-------------------------------------------------------------------------------------------

# Using Huggingface Pipeline for summarization models
summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")

print("Model-3 : google/pegasus-cnn_dailymail")
for example in dataset:
  # Accessing every Article and Summary from the dataset
  article = example["article"]
  summary = example["highlights"]

  # Generating Summary using the pipeline with max & min length of 150 & 30 respectively
  generated_summary = summarizer(article, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]
  print("Dataset Summary: ", summary)
  print("Generated Summary: ", generated_summary)

  # Initializing ROUGE Scorer for evaluation
  rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  # Calculating ROUGE scores of Dataset Summary and Generated Summary
  scores = rouge.score(summary, generated_summary)

  # Printing Precision, Recall & F1 scores of ROUGE-1, ROUGE-2 & ROUGE-L
  print("--------------------------------------------------------------------------------------------------------------------------")
  print("ROUGE-1:", scores['rouge1'])
  print("ROUGE-2:", scores['rouge2'])
  print("ROUGE-L:", scores['rougeL'])
  print("--------------------------------------------------------------------------------------------------------------------------")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model-3 : google/pegasus-cnn_dailymail
Dataset Summary:  Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated Summary:  The Palestinian Authority formally becomes the 123rd member of the International Criminal Court .<n>The move gives the court jurisdiction over alleged crimes in Palestinian territories .<n>Palestinians signed the ICC's founding Rome Statute in January .
--------------------------------------------------------------------------------------------------------------------------
ROUGE-1: Score(precision=0.3783783783783784, recall=0.4117647058823529, fmeasure=0.39436619718309857)
ROUGE-2: Score(precision=0.2222222222222222, recall=0.24242424242424243, fmeasure=0.2318840579710145)
ROUGE-L: Score(precision=0.2702702702702703, recall=0.29411764705882354, fmeasure=0.28169014084507044)
-----

Your max_length is set to 150, but your input_length is only 141. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)


Dataset Summary:  Once a super typhoon, Maysak is now a tropical storm with 70 mph winds .
It could still cause flooding, landslides and other problems in the Philippines .
Generated Summary:  Tropical storm Maysak is centered 200 miles southwest of Aurora province .<n>It's expected to make landfall Sunday morning on the southeastern coast of Isabela province .
--------------------------------------------------------------------------------------------------------------------------
ROUGE-1: Score(precision=0.2222222222222222, recall=0.23076923076923078, fmeasure=0.22641509433962265)
ROUGE-2: Score(precision=0.07692307692307693, recall=0.08, fmeasure=0.0784313725490196)
ROUGE-L: Score(precision=0.14814814814814814, recall=0.15384615384615385, fmeasure=0.1509433962264151)
--------------------------------------------------------------------------------------------------------------------------
Dataset Summary:  Bob Barker returned to host "The Price Is Right" on Wednesday .
Barker, 91, ha